In [54]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
from datetime import datetime

import matplotlib.pyplot as plt

In [170]:
year_path = [(2018, '/datadrive/timelapse_images/')]#] for y in range(2011, 2018)]

In [171]:
df = pd.DataFrame()
for y, _ in year_path:
    path = '/datadrive/timeseries_derived_data_products/MH11_resistivity_rock_{}.csv'.format(y)
    df = pd.concat((df, pd.read_csv(path)), axis=0)

In [172]:
df2 = pd.DataFrame()
for y, _ in year_path:
    path2 = '/datadrive/timeseries_derived_data_products/MH25_vaisalawxt520prec_{}.csv'.format(y)
    df2 = pd.concat((df2, pd.read_csv(path2)), axis=0)

In [173]:
df3 = pd.DataFrame()
for y, _ in year_path:
    path3 = '/datadrive/timeseries_derived_data_products/MH25_vaisalawxt520windpth_2017.csv'.format(y)
    df3 = pd.concat((df3, pd.read_csv(path3)), axis=0)

In [174]:
df.head()

,time,position [],device_id [],resistivity_10cm [Mohm],resistivity_35cm [Mohm],resistivity_60cm [Mohm],resistivity_85cm [Mohm],resistivity_10_35cm [Mohm],resistivity_35_60cm [Mohm],resistivity_60_85cm [Mohm],resistivity_10_60cm [Mohm],resistivity_35_85cm [Mohm],resistivity_10_85cm [Mohm],ref1 [°C],ref2 [°C],ref3 [°C],ref4 [°C],ref5 [°C]
0,2017-01-01T00:00:00Z,11.0,2096.0,1599.0,962.1608,127.0015,7.8923,236.2396,27.6041,14.1885,1809.5477,277.4623,1860.5342,1.8808,-9.7943,1.0,1.8808,-9.7943
1,2017-01-01T01:00:00Z,11.0,2096.0,1599.0,965.4289,127.5495,7.8979,238.6480,27.6599,14.2005,1806.2707,280.4639,1862.2691,1.8808,-9.7944,1.0,1.8805,-9.7945
2,2017-01-01T02:00:00Z,11.0,2096.0,1599.0,968.6970,128.1990,7.9034,240.3905,27.7267,14.2107,1817.4124,282.5258,1881.3530,1.8809,-9.7942,1.0,1.8807,-9.7941
3,2017-01-01T03:00:00Z,11.0,2096.0,1599.0,977.8403,129.2797,7.9050,242.2904,27.7779,14.2212,1822.6555,284.6370,1876.1483,1.8813,-9.7939,1.0,1.8813,-9.7942
4,2017-01-01T04:00:00Z,11.0,2096.0,1599.0,982.1713,130.1579,7.9075,243.8829,27.8383,14.2247,1821.0171,286.5846,1877.8832,1.8813,-9.7939,1.0,1.8813,-9.7942


In [175]:
def interpolate_to_timestamps(df, time_stamps):
    df = df.resample('4T').interpolate()
    ind = [df.index.get_loc(tim, method='nearest') for tim in time_stamps.time]
    return df.iloc[ind, :]

In [176]:
def extract_time_stamps(year_path):
    """
    input: [(year, root_dir), ...]
    """
    dfs = pd.DataFrame()
    for y, root_dir in year_path:
        path_dict = {}
        for filename in glob.iglob(root_dir + '**/'+str(y)+'*/*', recursive=True):
            di, filname = os.path.split(filename)
            ddi, ydi = os.path.split(di)
            path_dict[filname] = extract_time(filname)
        df = pd.DataFrame({'time':list(path_dict.values()), 'filename': list(path_dict.keys())}, )
        dfs = pd.concat((dfs, df), axis=0)
    return dfs

In [177]:
def extract_time(filname):
    return datetime.strptime(filname, '%Y%m%d_%H%M%S.JPG')

In [178]:
def extract_summer_days(time_stamps):
    start = pd.Timestamp(datetime(year=2000, month=1, day=1, hour=8)).time()
    end = pd.Timestamp(datetime(year=2000, month=1, day=1, hour=20)).time()
    time_stamps_day = time_stamps[np.logical_and(time_stamps['time'].dt.time >= start , 
                                                 time_stamps['time'].dt.time <= end)]
    
    june = pd.Timestamp(datetime(year=2000, month=5, day=1, hour=8)).month
    august = pd.Timestamp(datetime(year=2000, month=8, day=1, hour=8)).month

    time_stamps_summerday = time_stamps_day[np.logical_and(time_stamps['time'].dt.month < august , 
                                                           time_stamps['time'].dt.month >= june)]
    
    return time_stamps_summerday


In [179]:
#time_stamps = pd.read_pickle('pd_time_stamps.pkl')
#time_stamps.head()
time_stamps = extract_time_stamps(year_path)
summer_days = extract_summer_days(time_stamps)
time_stamps.head()

/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,filename,time
0,20170205_100011.JPG,2017-02-05 10:00:11
1,20170709_220006.JPG,2017-07-09 22:00:06
2,20170624_100005.JPG,2017-06-24 10:00:05
3,20170529_220006.JPG,2017-05-29 22:00:06
4,20170815_120013.JPG,2017-08-15 12:00:13


In [180]:
#summer_days = pd.read_pickle('pd_time_stamps_summerdays.pkl')

In [181]:
df = df.set_index(pd.DatetimeIndex(df.loc[:, 'time']))
df_interp = interpolate_to_timestamps(df, summer_days)

/anaconda/envs/py35/lib/python3.5/site-packages/pandas/core/indexes/base.py:3071: FutureWarning: Converting timezone-aware DatetimeArray to timezone-naive ndarray with 'datetime64[ns]' dtype. In the future, this will return an ndarray with 'object' dtype where each element is a 'pandas.Timestamp' with the correct 'tz'.
	To accept the future behavior, pass 'dtype=object'.
	To keep the old behavior, pass 'dtype="datetime64[ns]"'.
  target = np.asarray(target)


In [182]:
df2 = df2.set_index(pd.DatetimeIndex(df2.loc[:, 'time']))
df2_interp = interpolate_to_timestamps(df2, summer_days)

/anaconda/envs/py35/lib/python3.5/site-packages/pandas/core/indexes/base.py:3071: FutureWarning: Converting timezone-aware DatetimeArray to timezone-naive ndarray with 'datetime64[ns]' dtype. In the future, this will return an ndarray with 'object' dtype where each element is a 'pandas.Timestamp' with the correct 'tz'.
	To accept the future behavior, pass 'dtype=object'.
	To keep the old behavior, pass 'dtype="datetime64[ns]"'.
  target = np.asarray(target)


In [183]:
#df3 = df3.set_index(pd.DatetimeIndex(df3.loc[:, 'time']))
#df3_interp = interpolate_to_timestamps(df3, summer_days)

In [184]:
#df_interp = df_interp.loc[~df_interp.index.duplicated(keep='first')]
#df2_interp = df2_interp.loc[~df2_interp.index.duplicated(keep='first')]
#df3_interp = df3_interp.loc[~df3_interp.index.duplicated(keep='first')]

#dataset = pd.concat((df_interp, df2_interp, df3_interp), axis=1)
df_interp['path'] = summer_days.filename.values
#dataset = dataset.rename(columns={'rain_intensity [mm/h]': 'rain_intensity'})

In [200]:
df_interp['label_thresh_rest10_1'] = df_interp.loc[:, 'resistivity_10cm [Mohm]'] < 300
df_interp['label_thresh_rest10_2'] = np.logical_and(300 < df_interp.loc[:, 'resistivity_10cm [Mohm]'],
                                                    df_interp.loc[:, 'resistivity_10cm [Mohm]'] < 1200)
df_interp['label_thresh_rest10_3'] = 1200 < df_interp.loc[:, 'resistivity_10cm [Mohm]']
df_interp['label_thresh_rest10'] = np.where(df_interp.loc[:, ['label_thresh_rest10_1', 
                                                      'label_thresh_rest10_2', 
                                                      'label_thresh_rest10_3']].values)[1]
print(df_interp)
df_interp['rain_label'] = df2_interp.loc[:, 'rain_intensity [mm/h]'] > 1

time
2017-06-24 10:00:00+00:00    0
2017-07-27 12:00:00+00:00    0
2017-07-27 18:00:00+00:00    0
2017-06-29 16:00:00+00:00    0
2017-06-08 10:00:00+00:00    0
                            ..
2017-07-21 14:00:00+00:00    0
2017-05-08 16:00:00+00:00    2
2017-07-12 18:00:00+00:00    0
2017-06-04 18:00:00+00:00    0
2017-07-11 08:00:00+00:00    0
Name: label_thresh_rest10, Length: 472, dtype: int64


In [193]:
df_interp.query('rain_label == False').loc[:, ['path', 'label_thresh_rest10']].to_csv('labels_rain_resist.csv', header=False)
df_interp.loc[:, ['path', 'label_thresh_rest10_1', 'label_thresh_rest10_2', 
                  'label_thresh_rest10_3']].to_csv('/datadrive/labels.csv', header=False)

/anaconda/envs/py35/lib/python3.5/site-packages/pandas/core/generic.py:456: FutureWarning: The default of the 'keep_tz' keyword in DatetimeIndex.to_series will change to True in a future release. You can set 'keep_tz=True' to obtain the future behaviour and silence this warning.
  s = level_values.to_series()
/anaconda/envs/py35/lib/python3.5/site-packages/pandas/core/generic.py:464: FutureWarning: The default of the 'keep_tz' keyword in DatetimeIndex.to_series will change to True in a future release. You can set 'keep_tz=True' to obtain the future behaviour and silence this warning.
  dindex = axis_index.to_series()


In [194]:
summer_days.filename

2       20170624_100005.JPG
16      20170727_120014.JPG
17      20170727_180012.JPG
18      20170629_160003.JPG
26      20170608_100005.JPG
               ...         
2846    20170721_140010.JPG
2849    20170508_160006.JPG
2850    20170712_180004.JPG
2856    20170604_180005.JPG
2866    20170711_080006.JPG
Name: filename, Length: 472, dtype: object

In [188]:
import io
import csv
import shutil
img_folder = '/datadrive/timelapse_images'
img_folder_test = '/datadrive/timelapse_images_test'

with open('/datadrive/labels.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        print(row)
        img_name = row[1]
        label = int(row[2]=='True')
        month_folder = row[0][:10]
        #print(month_folder)
        #print(img_name,label)
        print(os.path.join(img_folder,month_folder,img_name))
        shutil.copyfile(os.path.join(img_folder,month_folder,img_name),os.path.join(img_folder_test,str(label),img_name))

['2017-06-24 10:00:00+00:00', '20170624_100005.JPG', 'True']
/datadrive/timelapse_images/2017-06-24/20170624_100005.JPG
['2017-07-27 12:00:00+00:00', '20170727_120014.JPG', 'False']
/datadrive/timelapse_images/2017-07-27/20170727_120014.JPG
['2017-07-27 18:00:00+00:00', '20170727_180012.JPG', 'False']
/datadrive/timelapse_images/2017-07-27/20170727_180012.JPG
['2017-06-29 16:00:00+00:00', '20170629_160003.JPG', 'False']
/datadrive/timelapse_images/2017-06-29/20170629_160003.JPG
['2017-06-08 10:00:00+00:00', '20170608_100005.JPG', 'False']
/datadrive/timelapse_images/2017-06-08/20170608_100005.JPG
['2017-06-30 18:00:00+00:00', '20170630_180004.JPG', 'False']
/datadrive/timelapse_images/2017-06-30/20170630_180004.JPG
['2017-05-29 16:00:00+00:00', '20170529_160005.JPG', 'False']
/datadrive/timelapse_images/2017-05-29/20170529_160005.JPG
['2017-07-19 16:00:00+00:00', '20170719_160010.JPG', 'True']
/datadrive/timelapse_images/2017-07-19/20170719_160010.JPG
['2017-06-05 18:00:00+00:00', '201